### 모델 출력 테스트

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 필요한 라이브러리 설치
!pip install -q transformers torch accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

print("라이브러리 임포트 완료!")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

In [ ]:
# 모델 경로 설정
model_path = "/content/drive/MyDrive/Colab Notebooks/woke-odds/checkpoint-best"

# 토크나이저 로드
print("토크나이저 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 특수 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✓ 토크나이저 로드 완료!")

# 모델 로드
print("\n모델 로딩 중...")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

# 평가 모드로 설정
model.eval()

print(f"✓ 모델 로드 완료!")
print(f"Device: {model.device}")

In [ ]:
# 시스템 프롬프트 (데이터셋 원본)
SYSTEM_PROMPT = """You are an AI that generates a single, concise clarifying question when a user's query is ambiguous.

Task:
Generate exactly one clarifying question based on the ambiguity type.
If the query is clear and needs no clarification, output: <NO_CLARIFYING_QUESTION>

Output format: One clarifying question (or <NO_CLARIFYING_QUESTION> if not needed)

Categories:
- EM (Epistemic Misalignment): Questions with unfamiliar entities or self-contradictions
- LA (Linguistic Ambiguity): Questions with lexical or semantic ambiguity
- AO (Aleatoric Output): Questions with missing contextual information causing confusion
- NONE: Clear questions that don't require clarification

Subclasses:
For EM:
- UNF (UNFAMILIAR): Query contains unfamiliar entities or facts
- CONT (CONTRADICTION): Query contains self-contradictions

For LA:
- LEX (LEXICAL): Query contains terms with multiple meanings
- SEM (SEMANTIC): Query lacks context leading to multiple interpretations

For AO:
- WHOM: Query output contains confusion due to missing personal elements
- WHEN: Query output contains confusion due to missing temporal elements
- WHERE: Query output contains confusion due to missing spatial elements
- WHAT: Query output contains confusion due to missing task-specific elements

For Clear Questions:
- NONE: Use when require_clarification=0, output <NO_CLARIFYING_QUESTION>"""

print("시스템 프롬프트 정의 완료!")

In [ ]:
# 테스트 샘플 3개 (messages 형식으로)
test_samples = [
    {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": "[LA|LEX] What is the most common type of type used in printing?"}
        ],
        "ground_truth": "Are you referring to the most common type of printed characters used in printing, or the most common type of person involved in the printing industry?"
    },
    {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": "[AO|WHEN] Number of oil refineries in the united states?"}
        ],
        "ground_truth": "Which one: 2019, or January 2015?"
    },
    {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": "[NONE|NONE] What are the names of the actors who acted in the movie Padayappa?"}
        ],
        "ground_truth": "<NO_CLARIFYING_QUESTION>"
    }
]

print(f"테스트 샘플 {len(test_samples)}개 준비 완료!")

In [ ]:
print("=== 모델 출력 테스트 (3개 샘플) ===\n")

for idx, sample in enumerate(test_samples):
    messages = sample['messages']
    ground_truth = sample['ground_truth']

    # system + user 메시지만 사용 (친구 방식)
    input_messages = [msg for msg in messages if msg['role'] != 'assistant']

    # user query 추출 (출력용)
    user_query = [msg['content'] for msg in messages if msg['role'] == 'user'][0]

    # Chat template 적용
    prompt = tokenizer.apply_chat_template(
        input_messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # 디코딩 (입력 부분 제외)
    generated_text = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    )

    # 출력
    print(f"[샘플 {idx+1}]")
    print(f"User Query: {user_query}")
    print(f"\nGround Truth: {ground_truth}")
    print(f"\nModel Output: {generated_text.strip()}")
    print("\n" + "="*80 + "\n")

print("테스트 완료!")

### Temperature 달리해서 응답 5개씩 생성해보기

In [ ]:
def generate_multiple_candidates(messages, model, tokenizer, num_candidates=5, temperatures=[0.7, 0.9, 1.1, 1.3, 1.5]):
    """
    같은 입력에 대해 다양한 temperature로 여러 후보 응답 생성

    Args:
        messages: 입력 메시지 리스트
        model: 학습된 모델
        tokenizer: 토크나이저
        num_candidates: 생성할 후보 개수 (기본 5개)
        temperatures: 사용할 temperature 리스트

    Returns:
        candidates: 생성된 후보 응답 리스트
    """
    candidates = []

    # system + user 메시지만 사용
    input_messages = [msg for msg in messages if msg['role'] != 'assistant']

    # Chat template 적용
    prompt = tokenizer.apply_chat_template(
        input_messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    print(f"다양한 temperature로 {num_candidates}개 후보 생성 중...")

    # 각 temperature로 생성
    for i, temp in enumerate(temperatures[:num_candidates]):
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=temp,
                top_p=0.95,  # 다양성을 위해 top_p도 높게
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # 디코딩
        generated_text = tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        ).strip()

        candidates.append({
            'text': generated_text,
            'temperature': temp
        })

        print(f"  [{i+1}/{num_candidates}] T={temp:.1f}: {generated_text[:80]}...")

    # 중복 제거
    unique_candidates = []
    seen_texts = set()
    for cand in candidates:
        if cand['text'] not in seen_texts:
            unique_candidates.append(cand)
            seen_texts.add(cand['text'])

    print(f"✓ 총 {len(unique_candidates)}개 고유 후보 생성 완료 (중복 {len(candidates) - len(unique_candidates)}개 제거)\n")

    return unique_candidates

print("후보 생성 함수 정의 완료!")

In [ ]:
# 샘플 1개에 대해 테스트
print("="*80)
print("[샘플 1] 후보 생성 테스트")
print("="*80 + "\n")

sample = test_samples[0]
messages = sample['messages']
user_query = [msg['content'] for msg in messages if msg['role'] == 'user'][0]
ground_truth = sample['ground_truth']

print(f"User Query: {user_query}\n")
print(f"Ground Truth: {ground_truth}\n")
print("-"*80 + "\n")

# 5개 후보 생성 (temperature: 0.7, 0.9, 1.1, 1.3, 1.5)
candidates_sample1 = generate_multiple_candidates(
    messages=messages,
    model=model,
    tokenizer=tokenizer,
    num_candidates=5
)

print("\n" + "="*80)
print("생성된 후보 목록:")
print("="*80 + "\n")

for idx, cand in enumerate(candidates_sample1, 1):
    print(f"[후보 {idx}] (T={cand['temperature']:.1f})")
    print(f"{cand['text']}")
    print()

In [ ]:
# 모든 샘플에 대해 후보 생성
all_candidates = []

print("\n" + "="*80)
print("3개 샘플 전체에 대한 후보 생성 시작")
print("="*80 + "\n")

for idx, sample in enumerate(test_samples, 1):
    messages = sample['messages']
    user_query = [msg['content'] for msg in messages if msg['role'] == 'user'][0]

    print(f"\n{'='*80}")
    print(f"[샘플 {idx}] {user_query}")
    print('='*80 + "\n")

    # 후보 생성
    candidates = generate_multiple_candidates(
        messages=messages,
        model=model,
        tokenizer=tokenizer,
        num_candidates=5  # 샘플당 5개 후보
    )

    # 저장
    all_candidates.append({
        'sample_idx': idx,
        'user_query': user_query,
        'ground_truth': sample['ground_truth'],
        'category': user_query.split(']')[0] + ']',  # 카테고리 추출
        'candidates': candidates
    })

print("\n" + "="*80)
print(f"✓ 전체 후보 생성 완료! (총 {len(all_candidates)}개 샘플)")
print("="*80)

In [ ]:
# 생성된 모든 후보 출력
for sample_data in all_candidates:
    print("\n" + "="*80)
    print(f"[샘플 {sample_data['sample_idx']}]")
    print("="*80)
    print(f"Query: {sample_data['user_query']}")
    print(f"Category: {sample_data['category']}")
    print(f"Ground Truth: {sample_data['ground_truth']}")
    print(f"\n생성된 후보 {len(sample_data['candidates'])}개:")
    print("-"*80)

    for idx, cand in enumerate(sample_data['candidates'], 1):
        print(f"\n[후보 {idx}] (Temperature={cand['temperature']:.1f})")
        print(f"{cand['text']}")

    print("\n")

### 데이터셋 불러오기

In [ ]:
import json
import pandas as pd

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/woke-odds/clamber_benchmark.jsonl'

In [ ]:
data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if line:
            try:
                # 첫 번째 파싱: 바깥쪽 따옴표로 감싸진 문자열을 파싱
                parsed_once = json.loads(line)

                # 두 번째 파싱: 실제 JSON 객체로 파싱
                if isinstance(parsed_once, str):
                    item = json.loads(parsed_once)
                else:
                    item = parsed_once

                data.append(item)
            except json.JSONDecodeError as e:
                print(f"Line {line_num} - Error: {e}")
                print(f"Content preview: {line[:100]}...")
                continue

# DataFrame으로 변환
df = pd.DataFrame(data)

In [ ]:
print(f"총 데이터 개수: {len(df)}")
print(f"\n컬럼 목록:\n{df.columns.tolist()}")

In [ ]:
from IPython.display import display
display(df.head())

In [ ]:
# 필요한 컬럼만 선택
df_cleaned = df[['question', 'clarifying_question', 'require_clarification', 'category', 'subclass']].copy()

print(f"✓ 컬럼 선택 완료!")
print(f"총 데이터 개수: {len(df_cleaned)}")
print(f"컬럼 목록: {df_cleaned.columns.tolist()}")

print("\n결측치 확인:")
print(df_cleaned.isnull().sum())

print("\n첫 5개 샘플:")
print(display(df_cleaned.head()))

In [ ]:
# 필요한 컬럼만 선택
df_cleaned = df[['question', 'clarifying_question', 'require_clarification', 'category', 'subclass']].copy()

# Category 매핑 (FD->EM, MC->AO)
category_mapping = {
    'FD': 'EM',  # Epistemic Misalignment
    'MC': 'AO',  # Aleatoric Output
    'LA': 'LA'   # Linguistic Ambiguity (유지)
}

# Category 이름 변경
df_cleaned['category'] = df_cleaned['category'].replace(category_mapping)

# 모호하지 않은 질문(0)에 대해 카테고리 NONE으로 변경
df_cleaned.loc[df_cleaned['require_clarification'] == 0, 'category'] = 'NONE'

# Subclass 매핑
subclass_mapping = {
    'whom': 'WHOM',
    'what': 'WHAT',
    'when': 'WHEN',
    'where': 'WHERE',
    'NK': 'UNF',
    'ICL': 'CONT',
    'co-reference': 'SEM',
    'polysemy': 'LEX'
}

# Subclass 이름 변경
df_cleaned['subclass'] = df_cleaned['subclass'].replace(subclass_mapping)

# 모호하지 않은 질문(0)에 대해 서브클래스 NONE으로 변경
df_cleaned.loc[df_cleaned['require_clarification'] == 0, 'subclass'] = 'NONE'

print(f"✓ 매핑 완료!")
print(f"총 데이터 개수: {len(df_cleaned)}")

print("\n변경 후 카테고리 분포:")
print(df_cleaned['category'].value_counts())

print("\n변경 후 서브클래스 분포:")
print(df_cleaned['subclass'].value_counts())

print("\n첫 5개 샘플:")
display(df_cleaned.head())  # display만 단독으로 사용

print("\nrequire_clarification=0인 샘플 확인:")
display(df_cleaned[df_cleaned['require_clarification'] == 0].head())

In [ ]:
# user_query 생성: [category|subclass] question
df_cleaned['user_query'] = df_cleaned.apply(
    lambda row: f"[{row['category']}|{row['subclass']}] {row['question']}",
    axis=1
)

# ground_truth는 clarifying_question으로
df_cleaned['ground_truth'] = df_cleaned['clarifying_question']

# 필요한 컬럼만 남기기
df_final = df_cleaned[['user_query', 'ground_truth']].copy()

# NONE 케이스 마스크 (regex=False 추가!)
none_mask = df_final['user_query'].str.contains('[NONE|NONE]', regex=False)

# ground_truth를 <NO_CLARIFYING_QUESTION>으로 채우기
df_final.loc[none_mask, 'ground_truth'] = '<NO_CLARIFYING_QUESTION>'

print(f"✓ DataFrame 생성 완료!")
print(f"총 데이터 개수: {len(df_final)}")
print(f"NONE 데이터: {none_mask.sum()}개 ({none_mask.sum()/len(df_final)*100:.1f}%)")
print(f"모호한 질문: {(~none_mask).sum()}개 ({(~none_mask).sum()/len(df_final)*100:.1f}%)")

In [ ]:
# 랜덤으로 10개 샘플 확인
import random

sample_indices = random.sample(range(len(df_final)), 10)

for i, idx in enumerate(sample_indices, 1):
    row = df_final.iloc[idx]
    print(f"\n[샘플 {i}] (인덱스: {idx})")
    print("-"*80)
    print(f"User Query: {row['user_query']}")
    print(f"Ground Truth: {row['ground_truth']}")

In [ ]:
import numpy as np

# 모호한 질문과 명확한 질문 분리
ambiguous_df = df_final[~none_mask].copy()
none_df = df_final[none_mask].copy()

print(f"\n분리 전:")
print(f"- 모호한 질문: {len(ambiguous_df)}개")
print(f"- 명확한 질문 (NONE): {len(none_df)}개")

# NONE 데이터를 5%만 샘플링
target_none_count = int(len(ambiguous_df) * 0.05)  # 모호한 질문의 5%
none_sampled = none_df.sample(n=min(target_none_count, len(none_df)), random_state=42)

# 합치기
df_final = pd.concat([ambiguous_df, none_sampled], ignore_index=True)

# 섞기
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n샘플링 후:")
print(f"- 모호한 질문: {len(ambiguous_df)}개")
print(f"- 명확한 질문 (NONE): {len(none_sampled)}개 ({len(none_sampled)/len(df_final)*100:.1f}%)")
print(f"- 총 데이터: {len(df_final)}개")

print("\n컬럼 목록:", df_final.columns.tolist())
print("\n결측치 확인:")
print(df_final.isnull().sum())

In [ ]:
# 최종 분포 확인
print("="*80)
print("최종 데이터 분포:")
print("="*80)

none_final_mask = df_final['user_query'].str.contains('[NONE|NONE]', regex=False)
print(f"NONE: {none_final_mask.sum()}개 ({none_final_mask.sum()/len(df_final)*100:.1f}%)")
print(f"모호한 질문: {(~none_final_mask).sum()}개 ({(~none_final_mask).sum()/len(df_final)*100:.1f}%)")

print("\n" + "="*80)
print("NONE 샘플 3개:")
print("="*80)
display(df_final[none_final_mask].head(3))

print("\n" + "="*80)
print("모호한 질문 샘플 3개:")
print("="*80)
display(df_final[~none_final_mask].head(3))

In [ ]:
import json

# 저장 경로
save_path = '/content/drive/MyDrive/Colab Notebooks/woke-odds/dpo_base_data.jsonl'

# JSONL 저장
with open(save_path, 'w', encoding='utf-8') as f:
    for idx, row in df_final.iterrows():
        json_line = {
            'user_query': row['user_query'],
            'ground_truth': row['ground_truth']
        }
        f.write(json.dumps(json_line, ensure_ascii=False) + '\n')

print(f"✓ JSONL 저장 완료!")
print(f"저장 경로: {save_path}")
print(f"저장된 데이터 개수: {len(df_final)}개")

# 저장 확인
print("\n저장된 파일 크기 확인:")
import os
file_size = os.path.getsize(save_path)
print(f"파일 크기: {file_size / 1024:.2f} KB ({file_size / 1024 / 1024:.2f} MB)")

In [ ]:
# 저장된 파일 다시 불러와서 확인
print("="*80)
print("저장된 파일 확인:")
print("="*80)

test_data = []
with open(save_path, 'r', encoding='utf-8') as f:
    for line in f:
        test_data.append(json.loads(line))

print(f"불러온 데이터 개수: {len(test_data)}")
print("\n첫 3개 샘플:")
for i, sample in enumerate(test_data[:3], 1):
    print(f"\n[샘플 {i}]")
    print(f"User Query: {sample['user_query']}")
    print(f"Ground Truth: {sample['ground_truth']}")

print("\n✓ 파일이 정상적으로 저장되었습니다!")

### 후보 생성 테스트

In [ ]:
from tqdm import tqdm

def generate_candidates_for_query(user_query, model, tokenizer, num_candidates=5):
    """
    하나의 쿼리에 대해 여러 후보 생성

    Returns:
        list: 생성된 후보 텍스트 리스트
    """
    # messages 구성
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]

    # Chat template 적용
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    candidates = []
    temperatures = [0.7, 0.9, 1.1, 1.3, 1.5]

    for temp in temperatures[:num_candidates]:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=temp,
                top_p=0.95,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        generated_text = tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        ).strip()

        candidates.append(generated_text)

    # 중복 제거하되, 5개 유지
    unique_candidates = []
    seen = set()
    for cand in candidates:
        if cand not in seen:
            unique_candidates.append(cand)
            seen.add(cand)

    # 5개 미만이면 재생성 (높은 temperature로)
    while len(unique_candidates) < num_candidates:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=1.5 + len(unique_candidates) * 0.1,  # 점점 높임
                top_p=0.95,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        generated_text = tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        ).strip()

        if generated_text not in seen:
            unique_candidates.append(generated_text)
            seen.add(generated_text)

    return unique_candidates[:num_candidates]

print("후보 생성 함수 정의 완료!")

In [ ]:
# 먼저 3개만 테스트
print("3개 샘플로 테스트 시작...\n")

test_df = df_final.head(3).copy()

# 후보 컬럼 추가
for i in range(5):
    test_df[f'candidate_{i+1}'] = None

# 생성
for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="후보 생성 중"):
    user_query = row['user_query']

    try:
        candidates = generate_candidates_for_query(user_query, model, tokenizer)

        # DataFrame에 저장
        for i, cand in enumerate(candidates):
            test_df.at[idx, f'candidate_{i+1}'] = cand

    except Exception as e:
        print(f"\n에러 발생 (idx={idx}): {e}")
        continue

print("\n테스트 완료!")
print("\n결과 확인:")
print(test_df[['user_query', 'ground_truth', 'candidate_1', 'candidate_2']].head(3))

In [ ]:
# 한 샘플씩 세로로 보기 (더 깔끔함)
def show_sample(df, idx):
    """
    DataFrame의 특정 행을 보기 좋게 출력
    """
    row = df.iloc[idx]
    print("="*80)
    print(f"[샘플 {idx}]")
    print("="*80)
    for col in df.columns:
        print(f"\n{col}:")
        print(f"  {row[col]}")
    print("\n" + "="*80 + "\n")

# 사용
show_sample(test_df, 0)
show_sample(test_df, 1)
show_sample(test_df, 2)

### 3개 샘플 테스트

In [ ]:
# 3개 샘플만 추출
df_test = df_final.head(3).copy()

print("="*80)
print("테스트용 3개 샘플:")
print("="*80)
display(df_test)

# 후보 생성할 컬럼 추가 (5개)
for i in range(5):
    df_test[f'candidate_{i+1}'] = None

print("\n후보 생성 시작...\n")

# 각 샘플에 대해 5개 후보 생성
for idx in range(len(df_test)):
    row = df_test.iloc[idx]
    user_query = row['user_query']

    print(f"[{idx+1}/3] 생성 중: {user_query[:60]}...")

    # messages 구성
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]

    try:
        candidates = generate_candidates_for_query(user_query, model, tokenizer, num_candidates=5)

        # DataFrame에 저장
        for i, cand in enumerate(candidates):
            df_test.at[idx, f'candidate_{i+1}'] = cand

        print(f"  ✓ 완료!\n")

    except Exception as e:
        print(f"  ✗ 에러: {e}\n")

print("="*80)
print("후보 생성 완료!")
print("="*80)

In [ ]:
# Pandas 표시 옵션
pd.set_option('display.max_colwidth', 80)

print("\n생성된 DataFrame 구조:")
print(f"컬럼: {df_test.columns.tolist()}")
print(f"Shape: {df_test.shape}")

print("\n" + "="*80)
print("전체 DataFrame:")
print("="*80)
display(df_test)

In [ ]:
# 각 샘플을 예쁘게 출력
for idx in range(len(df_test)):
    row = df_test.iloc[idx]

    print("\n" + "="*80)
    print(f"[샘플 {idx+1}]")
    print("="*80)
    print(f"User Query:\n  {row['user_query']}\n")
    print(f"Ground Truth:\n  {row['ground_truth']}\n")
    print(f"생성된 후보 5개:")
    print("-"*80)

    for i in range(5):
        cand = row[f'candidate_{i+1}']
        print(f"\n[후보 {i+1}]")
        print(f"  {cand}")

    print("\n")

In [ ]:
# 테스트 df 삭제
del df_test
print("✓ 테스트 DataFrame 삭제 완료\n")

## 전체 데이터 작업

In [ ]:
# 전체 데이터로 작업
df_with_candidates = df_final.copy()

print("="*80)
print(f"전체 데이터 후보 생성 시작")
print("="*80)
print(f"총 데이터 개수: {len(df_with_candidates)}개")
print(f"예상 소요 시간: 약 2-3시간\n")

# 후보 생성할 컬럼 추가 (5개)
for i in range(5):
    df_with_candidates[f'candidate_{i+1}'] = None

# 배치 처리 (100개씩 중간 저장)
batch_size = 100
total_batches = (len(df_with_candidates) + batch_size - 1) // batch_size

for batch_idx in range(total_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(df_with_candidates))

    print(f"\n{'='*80}")
    print(f"[Batch {batch_idx+1}/{total_batches}] Processing {start_idx}-{end_idx}")
    print('='*80)

    for idx in tqdm(range(start_idx, end_idx), desc=f"Batch {batch_idx+1}"):
        row = df_with_candidates.iloc[idx]
        user_query = row['user_query']

        try:
            candidates = generate_candidates_for_query(user_query, model, tokenizer, num_candidates=5)

            # DataFrame에 저장
            for i, cand in enumerate(candidates):
                df_with_candidates.at[df_with_candidates.index[idx], f'candidate_{i+1}'] = cand

        except Exception as e:
            print(f"\n에러 발생 (idx={idx}): {e}")
            continue

    # 중간 저장 (100개마다)
    temp_save_path = f'/content/drive/MyDrive/Colab Notebooks/woke-odds/candidates_batch_{batch_idx+1}.csv'
    df_with_candidates.iloc[:end_idx].to_csv(temp_save_path, index=False, encoding='utf-8-sig')
    print(f"\n✓ 중간 저장 완료: {temp_save_path}")

print("\n" + "="*80)
print("✓ 전체 후보 생성 완료!")
print("="*80)

In [ ]:
# 최종 CSV 저장
final_save_path = '/content/drive/MyDrive/Colab Notebooks/woke-odds/dpo_with_candidates_full.csv'
df_with_candidates.to_csv(final_save_path, index=False, encoding='utf-8-sig')

print(f"✓ 최종 CSV 저장 완료!")
print(f"저장 경로: {final_save_path}")
print(f"저장된 데이터 개수: {len(df_with_candidates)}개")

# 통계
print("\n" + "="*80)
print("생성 통계:")
print("="*80)
for i in range(5):
    col = f'candidate_{i+1}'
    valid_count = df_with_candidates[col].notna().sum()
    print(f"{col}: {valid_count}/{len(df_with_candidates)} ({valid_count/len(df_with_candidates)*100:.1f}%)")

In [ ]:
# 랜덤 샘플 3개 확인
import random

sample_indices = random.sample(range(len(df_with_candidates)), 3)

for idx in sample_indices:
    row = df_with_candidates.iloc[idx]
    print("="*80)
    print(f"[샘플 {idx}]")
    print("="*80)
    print(f"User Query: {row['user_query']}")
    print(f"\nGround Truth: {row['ground_truth']}")
    print(f"\n생성된 후보들:")
    for i in range(5):
        print(f"  {i+1}. {row[f'candidate_{i+1}']}")
    print()

print(f"\n✓ 전체 작업 완료! 총 {len(df_with_candidates)}개 데이터 처리됨")